In [1]:
import os, os.path
import numpy as np
import pandas as pd
import setup_analysis as sa
import support_functions as sf
import importlib
importlib.reload(sa)
importlib.reload(sf)


<module 'support_functions' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py'>

In [27]:
fkdata = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/dev/fake_data/land_use_transition_baseline_assumption_fake_data.xlsx"
fp_mergetarg = os.path.join(sa.dir_out, "tmp_afolu_vars.csv")



df_mk = pd.read_excel(fkdata, sheet_name = "data_sheet")
fields_id = ["year", "cat"]
fields_dat = [x for x in df_mk.columns if (x not in fields_id + ["scen"])]
# set to long and get transition probabilities
df_mkl = pd.melt(df_mk, id_vars = fields_id, value_vars = fields_dat)
def format_prob(row):
    return "pij_%s_to_%s"%tuple(row)
df_mkl["field"] = np.array(df_mkl[["cat", "variable"]].apply(f, axis = 1))

# read dataset to merge into
df_mergetarg = pd.read_csv(fp_mergetarg)

df_mkw = pd.pivot(df_mkl[["year", "field", "value"]], index = "year", columns = "field", values = "value")
df_mkw.reset_index(drop = False, inplace = True)# = df_mkw.rename(columns = {"year": "time_period"})
df_mkw["time_period"] = df_mkw["year"].replace({2016:0, 2050:35})
fields_id = ["time_period"]
fields_dat = [x for x in df_mkw.columns if (x not in fields_id + ["field", "year"])]
df_mkw = df_mkw[fields_id + fields_dat].reset_index(drop = True)

df_mkw_out = pd.DataFrame({"time_period": list(range(36))})
df_mkw_out = pd.merge(df_mkw_out, df_mkw, how = "left")
df_mkw_out = df_mkw_out.interpolate().transpose().sort_index()
df_mkw_out.reset_index(drop = False, inplace = True)
df_mkw_out = df_mkw_out.rename(columns = {"index": "variable"})
df_mkw_out = df_mkw_out.rename(columns = dict(zip(df_mkw_out.columns, [str(x) for x in df_mkw_out.columns])))

df_mkw_out = pd.merge(df_mergetarg[["subsector", "src", "variable", "time_series_id", "strategy_id", "variable_trajectory_group", "variable_trajectory_group_trajectory_type", "normalize_group", "trajgroup_no_vary_q", "uniform_scaling_q", "min_35", "max_35"]], df_mkw_out).sort_values(by = ["variable"]).reset_index(drop = True)
df_mkw_out = pd.concat([df_mergetarg[~df_mergetarg["0"].isna()], df_mkw_out], axis = 0).sort_values(by = ["subsector", "variable"]).reset_index(drop = True) 
df_mkw_out.to_csv(os.path.join(sa.dir_out, "tmp_afolu_vars_with_mcmc.csv"), index = None, encoding = "UTF-8")


df_mkw_out2 = df_mkw_out[[str(x) for x in range(36)]].transpose().reset_index(drop = True)
var_fields = list(df_mkw_out["variable"])
df_mkw_out2 = df_mkw_out2.rename(columns = dict(zip([(x) for x in range(len(df_mkw_out2.columns))], var_fields)))
df_mkw_out2["time_period"] = list(range(len(df_mkw_out2)))
df_mkw_out2 = df_mkw_out2[["time_period"] + var_fields]

df_mkw_out2.to_csv(os.path.join(sa.dir_out, "fake_data_afolu.csv"), index = None, encoding = "UTF-8")

In [200]:
df_mkw_out = pd.read_csv(os.path.join(sa.dir_out, "tmp_afolu_vars_with_mcmc.csv"))
df_mkw_out2 = df_mkw_out[[str(x) for x in range(36)]].transpose().reset_index(drop = True)
var_fields = list(df_mkw_out["variable"])
df_mkw_out2 = df_mkw_out2.rename(columns = dict(zip([(x) for x in range(len(df_mkw_out2.columns))], var_fields)))
df_mkw_out2["time_period"] = list(range(len(df_mkw_out2)))
var_fields.sort()
df_mkw_out2 = df_mkw_out2[["time_period"] + var_fields]

df_mkw_out2.to_csv(os.path.join(sa.dir_out, "fake_data_afolu.csv"), index = None, encoding = "UTF-8")

In [190]:
#df_mkw_out2[[x for x in df_mkw_out2.columns if "pij_wetlands" in x]]
df_mkw_out[[("pij_wetlands" in x) for x in list(df_mkw_out["variable"])]]

,subsector,src,variable,time_series_id,strategy_id,variable_trajectory_group,variable_trajectory_group_trajectory_type,normalize_group,trajgroup_no_vary_q,uniform_scaling_q,...,26,27,28,29,30,31,32,33,34,35
197,Land Use,from_cr,pij_wetlands_to_croplands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005748,0.005748,0.005748,0.005748,0.005748,0.005748,0.005748,0.005748,0.005748,0.005748
198,Land Use,from_cr,pij_wetlands_to_forests_mangroves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005298,0.005298,0.005298,0.005298,0.005298,0.005298,0.005298,0.005298,0.005298,0.005298
199,Land Use,from_cr,pij_wetlands_to_forests_primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
200,Land Use,from_cr,pij_wetlands_to_forests_secondary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.015826,0.015826,0.015826,0.015826,0.015826,0.015826,0.015826,0.015826,0.015826,0.015826
201,Land Use,from_cr,pij_wetlands_to_grasslands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005838,0.005838,0.005838,0.005838,0.005838,0.005838,0.005838,0.005838,0.005838,0.005838
202,Land Use,from_cr,pij_wetlands_to_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.009665,0.009665,0.009665,0.009665,0.009665,0.009665,0.009665,0.009665,0.009665,0.009665
203,Land Use,from_cr,pij_wetlands_to_settlements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167
204,Land Use,from_cr,pij_wetlands_to_wetlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.957457,0.957457,0.957457,0.957457,0.957457,0.957457,0.957457,0.957457,0.957457,0.957457


In [191]:
df_mkw_out = pd.read_csv(os.path.join(sa.dir_out, "tmp_afolu_vars_with_mcmc.csv"))
df_mkw_out2 = df_mkw_out[[str(x) for x in range(36)]].transpose()#.reset_index(drop = True)

In [192]:
df_mkw_out2

,0,1,2,3,4,5,6,7,8,9,...,273,274,275,276,277,278,279,280,281,282
0,7.35,7.35,7.35,1.68,1.68,1.68,9.22,9.22,9.22,1.68,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
1,7.35,7.35,7.35,1.68,1.68,1.68,9.22,9.22,9.22,1.68,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
2,7.35,7.35,7.35,1.68,1.68,1.68,9.22,9.22,9.22,1.68,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
3,7.35,7.35,7.35,1.68,1.68,1.68,9.22,9.22,9.22,1.68,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
4,7.28,7.28,7.28,1.67,1.67,1.67,9.13,9.13,9.13,1.67,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
5,7.21,7.21,7.21,1.65,1.65,1.65,9.05,9.05,9.05,1.65,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
6,7.15,7.15,7.15,1.63,1.63,1.63,8.96,8.96,8.96,1.63,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
7,7.08,7.08,7.08,1.62,1.62,1.62,8.87,8.87,8.87,1.62,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
8,7.01,7.01,7.01,1.60,1.60,1.60,8.79,8.79,8.79,1.60,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
9,6.94,6.94,6.94,1.59,1.59,1.59,8.70,8.70,8.70,1.59,...,1.0,16.363636,9.090909,9.090909,0.0,0.909091,16.363636,9.545455,0.0,1.363636
